In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import puzz

In [35]:
net = puzz.Network(['a1', 'a2'])
net.add_obstacle('o1')
net.add_obstacle('o2')
net.add_obstacle('o3')
net.add_obstacle('o4')
net.add_ability('a3')
net.add_ability('a4')
net.nodes

set(['start'])
set(['start'])


{'a1': <puzz.Ability at 0x106864bd0>,
 'a2': <puzz.Ability at 0x106864c10>,
 'a3': <puzz.Ability at 0x106864cd0>,
 'a4': <puzz.Ability at 0x106864d10>,
 'o1': <puzz.Node at 0x106864b10>,
 'o2': <puzz.Node at 0x1067f6150>,
 'o3': <puzz.Node at 0x106864a90>,
 'o4': <puzz.Node at 0x106864b90>,
 'start': <puzz.Node at 0x106864b50>}

In [36]:
net.defeats('a1', 'o1')
net.defeats('a2', 'o2')
net.defeats('a3', 'o3')
net.defeats('a3', 'o1')
net.defeats('a4', 'o4')

In [37]:
net.calc_ability_prereqs()
net.nodes['a1'].reqs
net.nodes['a2'].reqs
net.nodes['a3'].reqs
net.nodes['a4'].reqs

set()

set()

{'a1'}

set()

In [38]:
# desired behavior: a3 obsoletes a1. so you should only be able to
# place it after a1, not a2
net.enabled_nodes('a1')
net.enabled_nodes('a2')

set(['a1', 'start'])
a1
set(['a1', 'start'])


{'a3', 'a4', 'o1', 'o3', 'o4'}

set(['start', 'a2'])
a1
a1 not in nodelist
set(['start', 'a2'])


{'a4', 'o1', 'o3', 'o4'}